## cross validation

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import pandas as pd
import tensorflow as tf
import theano
import keras 

import numpy as np
import matplotlib.pyplot as plt
from time import time

Using TensorFlow backend.


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8251680336568528501
]


### data preprocessing

In [3]:
df = pd.read_csv('../Artificial_Neural_Networks/Churn_Modelling.csv')
print df.shape
X = df.iloc[:,3:-1].values
y = df.iloc[:,-1].values

# Encoding the Independent Variable
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])

onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
print X.shape

# remove one dummy variable to avoid dummy variable trap
X = X[:, 1:]
print X.shape

(10000, 14)
(10000, 12)
(10000, 11)


### train test split & feature scaling 

In [4]:
S = lambda *x: [i.shape for i in x]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# print S(X_train,X_test, y_train, y_test)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
# fit on training set
X_train = sc.fit_transform(X_train)
# only transform on test set
X_test = sc.transform(X_test)
print S(X_train,X_test, y_train, y_test)

[(8000, 11), (2000, 11), (8000,), (2000,)]


### build k-fold func ANN
## Dropout and GridSearch (parameter tuning)

In [5]:
from keras.layers import Dropout

from keras.models import Sequential
from keras.layers import Dense

from keras.wrappers.scikit_learn import KerasClassifier
# k-fold cross validation
from sklearn.model_selection import GridSearchCV

In [6]:
def build_classifier(opt, P, h1, h2):
    classifier = Sequential()
    # first hidden layer
    # classifier.add(Dense(units = 6, input_shape=(11,),  kernel_initializer='uniform', activation='relu'))
    classifier.add(Dense(units = h1, input_dim=11,  kernel_initializer='uniform', activation='relu'))
    # Dropout
    classifier.add(Dropout(rate= P))
    
    # second hidden layer
    classifier.add(Dense(units = h2,  kernel_initializer='he_uniform', activation='relu'))
    # Dropout
    classifier.add(Dropout(rate= P))    
    
    # ouput layer
    classifier.add(Dense(units = 1,  kernel_initializer='he_uniform', activation='sigmoid'))
    # compiling the ANN
    classifier.compile(optimizer= opt, loss='binary_crossentropy', metrics= ['accuracy'])
#     print classifier.summary()
    return classifier

classifier = KerasClassifier(build_fn= build_classifier)

In [7]:
# parameters = {
#     'batch_size':[16, 32, 64],
#     'nb_epoch':[ 100, 300],
#     'opt':['adam'],
#     'P':[0.01],
#     'h1':[6, 12, 24],
#     'h2':[3, 6, 12]
# }

In [8]:
parameters = {
    'batch_size':[16, 32, 64],
    'nb_epoch':[ 100, 200],
    'opt':['adam'],
    'P':[0.01, 0.1],
    'h1':[24, 48, 96],
    'h2':[12, 48]
}

In [9]:
grid_sesarch = GridSearchCV(estimator= classifier, param_grid= parameters, 
                           scoring= 'accuracy', cv= 10, n_jobs=-1)

In [10]:
if __name__ == "__main__":
    import sys
    from time import time
    start = time()
    grid = grid_sesarch.fit(X_train, y_train)
    eclipse = time()-start
    print 'used: {:.2f} s'.format(eclipse)
    
    best_parameters = grid.best_params_
    best_accuracy = grid.best_score_

Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
7200/7200 [==============================] - 1s - loss: 0.4604 - acc: 0.8021     
Epoch 2/10
7200/7200 [==============================] - 1s - loss: 0.4624 - acc: 0.8008     
Epoch 2/10
7200/7200 [==============================] - 1s - loss: 0.4637 - acc: 0.8037     

Epoch 2/10
Epoch 2/10
7200/7200 [==============================] - 1s - loss: 0.3781 - acc: 0.8447     
Epoch 3/10
6816/7200 [===========================>..] - ETA: 0s - loss: 0.3697 - acc: 0.8490
Epoch 3/10
7200/7200 [==============================] - 1s - loss: 0.3684 - acc: 0.8500     
Epoch 3/10
6832/7200 [===========================>..] - ETA: 0s - loss: 0.3508 - acc: 0.8558
Epoch 4/10
7200/7200 [==============================] - 1s - loss: 0.3498 - acc: 0.8547     
Epoch 4/10
7200/7200 [==============================] - 1s - loss: 0.3504 - acc: 0.8557     
Epoch 4/10
7200/7200 [==============================] - 1s - loss: 0.3508 - acc: 0.8544     
Epoch 5/10
  16/7200 [...

7200/7200 [==============================] - 1s - loss: 0.3349 - acc: 0.8621     
Epoch 10/10
 288/7200 [>.............................] - ETA: 4s - loss: 0.6597 - acc: 0.7326 Epoch 1/10
Epoch 1/10
7200/7200 [==============================] - 1s - loss: 0.4464 - acc: 0.8124     
Epoch 2/10
7200/7200 [==============================] - 1s - loss: 0.4457 - acc: 0.8143     Epoch 2/10

1696/7200 [======>.......................] - ETA: 0s - loss: 0.3716 - acc: 0.8384
Epoch 2/10
4736/7200 [==================>...........] - ETA: 0s - loss: 0.3682 - acc: 0.8484
Epoch 3/10
7200/7200 [==============================] - 1s - loss: 0.3680 - acc: 0.8488     
Epoch 3/10
7200/7200 [==============================] - 1s - loss: 0.3719 - acc: 0.8490     
Epoch 3/10
7200/7200 [==============================] - 1s - loss: 0.3469 - acc: 0.8586     
Epoch 4/10
7200/7200 [==============================] - 1s - loss: 0.3527 - acc: 0.8590     
Epoch 4/10
 256/7200 [>.............................] - ETA: 1s - los

7200/7200 [==============================] - 1s - loss: 0.3370 - acc: 0.8612     
Epoch 9/10
7200/7200 [==============================] - 1s - loss: 0.3431 - acc: 0.8593     
Epoch 9/10
7200/7200 [==============================] - 1s - loss: 0.3430 - acc: 0.8574     
Epoch 10/10
3968/7200 [===============>..............] - ETA: 0s - loss: 0.3338 - acc: 0.8652
Epoch 10/10
7200/7200 [==============================] - 1s - loss: 0.3421 - acc: 0.8610     
Epoch 10/10
7200/7200 [==============================] - 1s - loss: 0.3395 - acc: 0.8621     
Epoch 10/10
6864/7200 [===========================>..] - ETA: 0sEpoch 1/10
Epoch 1/10
Epoch 1/10
7200/7200 [==============================] - 1s - loss: 0.4532 - acc: 0.8089     
Epoch 2/10
7200/7200 [==============================] - 1s - loss: 0.4508 - acc: 0.8094     
Epoch 2/10
7200/7200 [==============================] - 1s - loss: 0.4504 - acc: 0.8089     
Epoch 2/10
2896/7200 [===========>..................] - ETA: 0s - loss: 0.3788 - acc:

5888/7200 [=======================>......] - ETA: 0s - loss: 0.3444 - acc: 0.8592
Epoch 6/10
7200/7200 [==============================] - 1s - loss: 0.3442 - acc: 0.8586     
Epoch 7/10
7200/7200 [==============================] - 1s - loss: 0.3402 - acc: 0.8589     
Epoch 7/10
7200/7200 [==============================] - 1s - loss: 0.3459 - acc: 0.8558     
Epoch 7/10
7200/7200 [==============================] - 1s - loss: 0.3402 - acc: 0.8596     
Epoch 8/10
4976/7200 [===================>..........] - ETA: 0s - loss: 0.3457 - acc: 0.8585
Epoch 8/10
6400/7200 [=========================>....] - ETA: 0s - loss: 0.3384 - acc: 0.8614
Epoch 8/10
7200/7200 [==============================] - 1s - loss: 0.3358 - acc: 0.8626     
Epoch 8/10
7200/7200 [==============================] - 1s - loss: 0.3373 - acc: 0.8642     
Epoch 9/10
7200/7200 [==============================] - 1s - loss: 0.3436 - acc: 0.8581     
Epoch 9/10
7200/7200 [==============================] - 1s - loss: 0.3379 - acc: 

7200/7200 [==============================] - 1s - loss: 0.3670 - acc: 0.8492     
Epoch 3/10
7200/7200 [==============================] - 1s - loss: 0.3410 - acc: 0.8615     
Epoch 5/10
7200/7200 [==============================] - 1s - loss: 0.3496 - acc: 0.8582     
Epoch 4/10
7200/7200 [==============================] - 1s - loss: 0.3470 - acc: 0.8583     
Epoch 5/10
7200/7200 [==============================] - 1s - loss: 0.3488 - acc: 0.8578     
Epoch 5/10
3056/7200 [===========>..................] - ETA: 0s - loss: 0.3487 - acc: 0.8570
Epoch 6/10
7200/7200 [==============================] - 1s - loss: 0.3448 - acc: 0.8597     
Epoch 5/10
7200/7200 [==============================] - 1s - loss: 0.3456 - acc: 0.8565     
Epoch 6/10
7200/7200 [==============================] - 1s - loss: 0.3356 - acc: 0.8639     
Epoch 7/10
7200/7200 [==============================] - 1s - loss: 0.3420 - acc: 0.8574     
Epoch 6/10
7200/7200 [==============================] - 1s - loss: 0.3416 - acc: 

...

7200/7200 [==============================] - 0s - loss: 0.3666 - acc: 0.8508     
Epoch 5/10
7200/7200 [==============================] - 0s - loss: 0.3470 - acc: 0.8572     
Epoch 6/10
7200/7200 [==============================] - 0s - loss: 0.4024 - acc: 0.8313     
Epoch 6/10

Epoch 3/10
7200/7200 [==============================] - 0s - loss: 0.3478 - acc: 0.8579     
Epoch 7/10
7200/7200 [==============================] - 0s - loss: 0.3443 - acc: 0.8606     
Epoch 8/10
7200/7200 [==============================] - 0s - loss: 0.3584 - acc: 0.8539     
Epoch 5/10
7200/7200 [==============================] - 0s - loss: 0.3543 - acc: 0.8550     
Epoch 8/10
7200/7200 [==============================] - 0s - loss: 0.3434 - acc: 0.8607     
Epoch 9/10
7200/7200 [==============================] - 0s - loss: 0.3554 - acc: 0.8567     
Epoch 6/10
6400/7200 [=========================>....] - ETA: 0s - loss: 0.3541 - acc: 0.8544
Epoch 9/10
2304/7200 [========>.....................] - ETA: 0s - los

7200/7200 [==============================] - 0s - loss: 0.3714 - acc: 0.8464     
Epoch 4/10
7200/7200 [==============================] - 0s - loss: 0.3574 - acc: 0.8531     
Epoch 5/10
7200/7200 [==============================] - 4s - loss: 0.5082 - acc: 0.7840     
Epoch 2/10
7200/7200 [==============================] - 0s - loss: 0.3545 - acc: 0.8550     
Epoch 6/10
7200/7200 [==============================] - 0s - loss: 0.4002 - acc: 0.8306     
Epoch 3/10
3776/7200 [==============>...............] - ETA: 0s - loss: 0.3406 - acc: 0.8588
Epoch 4/10
7200/7200 [==============================] - 0s - loss: 0.3479 - acc: 0.8574     
Epoch 8/10
7200/7200 [==============================] - 0s - loss: 0.3601 - acc: 0.8538     
Epoch 5/10
7200/7200 [==============================] - 4s - loss: 0.5110 - acc: 0.7833     
Epoch 2/10
7200/7200 [==============================] - 0s - loss: 0.3571 - acc: 0.8549     
Epoch 5/10
7200/7200 [==============================] - 0s - loss: 0.3429 - acc: 

7200/7200 [==============================] - 0s - loss: 0.3385 - acc: 0.8589     
Epoch 9/10
7200/7200 [==============================] - 0s - loss: 0.3395 - acc: 0.8625     
Epoch 10/10
7200/7200 [==============================] - 4s - loss: 0.4754 - acc: 0.8006     
Epoch 2/10
7200/7200 [==============================] - 0s - loss: 0.3928 - acc: 0.8336     
Epoch 3/10
7200/7200 [==============================] - 0s - loss: 0.3657 - acc: 0.8514     
Epoch 4/10
7200/7200 [==============================] - 0s - loss: 0.3557 - acc: 0.8538     
Epoch 5/10
7200/7200 [==============================] - 0s - loss: 0.3512 - acc: 0.8574     
Epoch 6/10
7200/7200 [==============================] - 4s - loss: 0.4844 - acc: 0.7806     
Epoch 2/10
7200/7200 [==============================] - 0s - loss: 0.3462 - acc: 0.8614     
Epoch 7/10
7200/7200 [==============================] - 0s - loss: 0.3885 - acc: 0.8393     
Epoch 3/10
7200/7200 [==============================] - 0s - loss: 0.3596 - acc:

2240/7200 [========>.....................] - ETA: 0s - loss: 0.3388 - acc: 0.8625
Epoch 5/10
4928/7200 [===================>..........] - ETA: 0s - loss: 0.3497 - acc: 0.8551
Epoch 8/10
7200/7200 [==============================] - 0s - loss: 0.3482 - acc: 0.8574     
Epoch 6/10
7200/7200 [==============================] - 0s - loss: 0.3425 - acc: 0.8569     
Epoch 9/10
7200/7200 [==============================] - 0s - loss: 0.3442 - acc: 0.8582     
Epoch 7/10
7200/7200 [==============================] - 0s - loss: 0.3405 - acc: 0.8582     
Epoch 10/10
7200/7200 [==============================] - 0s - loss: 0.3451 - acc: 0.8589     
Epoch 8/10
7200/7200 [==============================] - 0s - loss: 0.3418 - acc: 0.8600     
Epoch 9/10
7200/7200 [==============================] - 0s - loss: 0.3406 - acc: 0.8611     
Epoch 10/10
7200/7200 [==============================] - 0s - loss: 0.3374 - acc: 0.8617     
Epoch 1/10
7200/7200 [==============================] - 4s - loss: 0.4639 - acc

7200/7200 [==============================] - 0s - loss: 0.3395 - acc: 0.8608     
Epoch 9/10
7200/7200 [==============================] - 0s - loss: 0.3473 - acc: 0.8547     
Epoch 5/10
7200/7200 [==============================] - 0s - loss: 0.3405 - acc: 0.8601     
Epoch 6/10
7200/7200 [==============================] - 0s - loss: 0.3392 - acc: 0.8626     
Epoch 10/10
7200/7200 [==============================] - 0s - loss: 0.3662 - acc: 0.8482     
Epoch 4/10
7200/7200 [==============================] - 0s - loss: 0.3405 - acc: 0.8590     
Epoch 7/10
7200/7200 [==============================] - 0s - loss: 0.3539 - acc: 0.8535     
Epoch 5/10
7200/7200 [==============================] - 0s - loss: 0.3392 - acc: 0.8618     
Epoch 8/10
2560/7200 [=========>....................] - ETA: 0s - loss: 0.3566 - acc: 0.8527
Epoch 6/10
7200/7200 [==============================] - 0s - loss: 0.3340 - acc: 0.8618     
Epoch 9/10
7200/7200 [==============================] - 0s - loss: 0.3348 - acc:

In [11]:
best_parameters

{'P': 0.1,
 'batch_size': 16,
 'h1': 48,
 'h2': 12,
 'nb_epoch': 100,
 'opt': 'adam'}

In [12]:
best_accuracy

0.86375000000000002

In [13]:
# GridSearchCV?

In [14]:
eclipse

2993.88751912117

In [15]:
from time import ctime
with open('result_6.txt', 'w+') as fl:
    fl.write(str(best_parameters)+'\n')
    fl.write(str(best_accuracy)+'\n')
    fl.write(str(eclipse)+'\n')
    fl.write(str(ctime())+'\n')

In [23]:

cls = build_classifier('adam', 0.1, 48, 12)
cls.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 48)                576       
_________________________________________________________________
dropout_3 (Dropout)          (None, 48)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 12)                588       
_________________________________________________________________
dropout_4 (Dropout)          (None, 12)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 13        
Total params: 1,177
Trainable params: 1,177
Non-trainable params: 0
_________________________________________________________________


In [24]:
from time import time, ctime
start = time()
# fitting ANN with training set
cls.fit(X_train, y_train, batch_size=16, epochs=100)
eclipse = time()-start
print ctime()
eclipse

Epoch 1/100
8000/8000 [==============================] - 0s - loss: 0.4359 - acc: 0.7960     
Epoch 2/100
8000/8000 [==============================] - 0s - loss: 0.3912 - acc: 0.8247     
Epoch 3/100
8000/8000 [==============================] - 0s - loss: 0.3680 - acc: 0.8490     
Epoch 4/100
8000/8000 [==============================] - 0s - loss: 0.3604 - acc: 0.8538     
Epoch 5/100
8000/8000 [==============================] - 0s - loss: 0.3552 - acc: 0.8550     
Epoch 6/100
8000/8000 [==============================] - 0s - loss: 0.3489 - acc: 0.8594     
Epoch 7/100
8000/8000 [==============================] - 0s - loss: 0.3504 - acc: 0.8596     
Epoch 8/100
8000/8000 [==============================] - 0s - loss: 0.3454 - acc: 0.8620     
Epoch 9/100
8000/8000 [==============================] - 0s - loss: 0.3479 - acc: 0.8564     
Epoch 10/100
8000/8000 [==============================] - 0s - loss: 0.3452 - acc: 0.8576     
Epoch 11/100
8000/8000 [==============================] - 0

8000/8000 [==============================] - 0s - loss: 0.3156 - acc: 0.8696     
Epoch 88/100
8000/8000 [==============================] - 0s - loss: 0.3167 - acc: 0.8728     
Epoch 89/100
8000/8000 [==============================] - 0s - loss: 0.3169 - acc: 0.8721     
Epoch 90/100
8000/8000 [==============================] - 0s - loss: 0.3169 - acc: 0.8746     
Epoch 91/100
8000/8000 [==============================] - 0s - loss: 0.3164 - acc: 0.8719     
Epoch 92/100
8000/8000 [==============================] - 0s - loss: 0.3127 - acc: 0.8715     
Epoch 93/100
8000/8000 [==============================] - 0s - loss: 0.3135 - acc: 0.8732     
Epoch 94/100
8000/8000 [==============================] - 0s - loss: 0.3145 - acc: 0.8708     
Epoch 95/100
8000/8000 [==============================] - 0s - loss: 0.3164 - acc: 0.8718     
Epoch 96/100
8000/8000 [==============================] - 0s - loss: 0.3118 - acc: 0.8756     
Epoch 97/100
8000/8000 [==============================] - 0s - 

52.89596676826477

In [25]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, cls.predict(X_test)>0.5)

0.85499999999999998

In [19]:
# if __name__ == "__main__":
#     from time import time
#     start=time()
#     accuracies = cross_val_score(estimator = classifier, X=X_train, y=y_train, cv=10, n_jobs=-1)
#     eclipse = time()-start
#     print 'used: {:.2f} s'.format(eclipse)

In [20]:
# accuracies

In [21]:
# accuracies.mean()

In [22]:
# accuracies.std()

### Predicting a single new observation